In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
import model
import data_process
import train_and_evaluate
import importlib

importlib.reload(model)
importlib.reload(data_process)
importlib.reload(train_and_evaluate)

# 重新加载的模块中导入所需的类和函数
from model import MultiModalModel
from data_process import TextEyeTrackingDataset
from train_and_evaluate import train_model, evaluate_model

In [3]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader
# 加载数据
dataset = TextEyeTrackingDataset('data/raw_data_0317.csv')
batch_size = 1

# 假设dataset是你已经加载的完整数据集
train_size = int(len(dataset) * 0.8)  # 以80%的数据作为训练集
test_size = len(dataset) - train_size  # 剩余20%的数据作为测试集
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
import torch.nn as nn

vocab_size = 10000  # 假设词汇表大小为10000
embedding_dim = 300  # 假设词嵌入维度为300
hidden_dim = 128  # LSTM隐藏层维度
output_dim = 1  # 输出维度，二分类问题

model = MultiModalModel(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)
optimizer = torch.optim.Adam(model.parameters())

pos_weight = torch.tensor([10])  # your_value是一个大于1的值，表示正类相对于负类的权重
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)


# 训练模型
train_model(model, train_loader, test_loader, optimizer, criterion, device, epochs=30)

/usr/local/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1, Loss: 0.5471951049583698, Accuracy: 0.98, AUC: 0.5, Precision: 0.0, Recall: 0.0


KeyboardInterrupt: 